In [1]:
import sqlalchemy

In [2]:
sqlalchemy.__version__

'1.2.16'

## Create an engine - interface to the database

In [3]:
from sqlalchemy import create_engine

In [4]:
engine = create_engine('sqlite:///:memory:', echo=True)

## Create Base - base class holding all the mappings between the Python code and database tables

In [5]:
from sqlalchemy.ext.declarative import declarative_base

In [6]:
Base = declarative_base()

## Create a User entity

In [7]:
from sqlalchemy import Column, Integer, String

In [8]:
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    name = Column(String(50))
    fullname = Column(String(50))
    password = Column(String(12))
    
    def __repr__(self):
        return f'<User(name={self.name}, fullname={self.fullname}, password={self.password})>'

In [9]:
Base.metadata.create_all(engine)

2019-01-20 11:18:13,761 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-01-20 11:18:13,762 INFO sqlalchemy.engine.base.Engine ()
2019-01-20 11:18:13,764 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-01-20 11:18:13,765 INFO sqlalchemy.engine.base.Engine ()
2019-01-20 11:18:13,767 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-01-20 11:18:13,767 INFO sqlalchemy.engine.base.Engine ()
2019-01-20 11:18:13,772 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR(50), 
	fullname VARCHAR(50), 
	password VARCHAR(12), 
	PRIMARY KEY (id)
)


2019-01-20 11:18:13,773 INFO sqlalchemy.engine.base.Engine ()
2019-01-20 11:18:13,775 INFO sqlalchemy.engine.base.Engine COMMIT


In [10]:
ed_user = User(name='ed', fullname='Ed Jones', password='edspassword')

In [11]:
ed_user.name

'ed'

In [12]:
str(ed_user.id)

'None'

## Create a session object (not instantiated)

In [13]:
from sqlalchemy.orm import sessionmaker

In [14]:
Session = sessionmaker(bind=engine) # this adds the interface to the database straight away

but we can also use
```Session = sessionmaker()```
and then add the engine later with:
```Session.configure(bind=engine)```
this is useful for using in app factory where we do not want to bind it initially.


## Create a session instance

In [15]:
session = Session()

## Add object to database

In [16]:
session.add(ed_user)

Now the ed_user is **pending** - its inside a transaction in the session waiting for commit() to be saved to the database.

## Retrieving pending object will flush it giving it a unique ID

In [17]:
our_user = session.query(User).filter_by(name='ed').first()

2019-01-20 11:18:13,859 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-20 11:18:13,862 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2019-01-20 11:18:13,863 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones', 'edspassword')
2019-01-20 11:18:13,866 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2019-01-20 11:18:13,867 INFO sqlalchemy.engine.base.Engine ('ed', 1, 0)


## The object retrieved from the database is the same object that was used to create it!

In [18]:
our_user is ed_user

True

## Add multiple objects with add_all()

In [19]:
session.add_all(
    [
        User(name='wendy', fullname='Wendy Williams', password='foobar'),
        User(name='mary', fullname='Mary Contrary', password='xfsdgfsd'),
        User(name='fred', fullname='Fred Flinstone', password='club'),
    ]
)

## Change existing attribute

In [20]:
ed_user.password = 'newpass'

## Now the changes to already existing objects are in ```session.dirty```

In [21]:
session.dirty

IdentitySet([<User(name=ed, fullname=Ed Jones, password=newpass)>])

## And new items that are pending are in ```session.new```

In [22]:
session.new

IdentitySet([<User(name=wendy, fullname=Wendy Williams, password=foobar)>, <User(name=mary, fullname=Mary Contrary, password=xfsdgfsd)>, <User(name=fred, fullname=Fred Flinstone, password=club)>])

## To save all pending changes we issue ```session.commit()``` command

In [23]:
session.commit()

2019-01-20 11:18:13,939 INFO sqlalchemy.engine.base.Engine UPDATE users SET password=? WHERE users.id = ?
2019-01-20 11:18:13,940 INFO sqlalchemy.engine.base.Engine ('newpass', 1)
2019-01-20 11:18:13,942 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2019-01-20 11:18:13,942 INFO sqlalchemy.engine.base.Engine ('wendy', 'Wendy Williams', 'foobar')
2019-01-20 11:18:13,943 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2019-01-20 11:18:13,945 INFO sqlalchemy.engine.base.Engine ('mary', 'Mary Contrary', 'xfsdgfsd')
2019-01-20 11:18:13,946 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2019-01-20 11:18:13,948 INFO sqlalchemy.engine.base.Engine ('fred', 'Fred Flinstone', 'club')
2019-01-20 11:18:13,949 INFO sqlalchemy.engine.base.Engine COMMIT


In [24]:
ed_user.id

2019-01-20 11:18:13,957 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-20 11:18:13,959 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.id = ?
2019-01-20 11:18:13,960 INFO sqlalchemy.engine.base.Engine (1,)


1

In [25]:
ed_user.password

'newpass'

## How to rollback the changes that are pending (not commited yet).

In [26]:
ed_user.name = 'Eduardo'

In [27]:
fake_user = User(name='fakeuser', fullname='Invalid', password='fsdfsd')

In [28]:
session.add(fake_user)

In [29]:
fake_user in session

True


## Query the session to flush the pending changes

In [30]:
session.query(User).filter(User.name.in_(['Eduardo', 'fakeuser'])).all()

2019-01-20 11:18:14,034 INFO sqlalchemy.engine.base.Engine UPDATE users SET name=? WHERE users.id = ?
2019-01-20 11:18:14,035 INFO sqlalchemy.engine.base.Engine ('Eduardo', 1)
2019-01-20 11:18:14,038 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2019-01-20 11:18:14,038 INFO sqlalchemy.engine.base.Engine ('fakeuser', 'Invalid', 'fsdfsd')
2019-01-20 11:18:14,041 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?, ?)
2019-01-20 11:18:14,041 INFO sqlalchemy.engine.base.Engine ('Eduardo', 'fakeuser')


[<User(name=Eduardo, fullname=Ed Jones, password=newpass)>,
 <User(name=fakeuser, fullname=Invalid, password=fsdfsd)>]

## Now rollback to revert the changes

In [31]:
session.rollback()

2019-01-20 11:18:14,051 INFO sqlalchemy.engine.base.Engine ROLLBACK


In [32]:
ed_user.name # the name is again 'ed'

2019-01-20 11:18:14,062 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-20 11:18:14,064 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.id = ?
2019-01-20 11:18:14,066 INFO sqlalchemy.engine.base.Engine (1,)


'ed'

In [33]:
fake_user in session # and the fake_user got removed from the session transaction

False

# Querying the database 

In [34]:
for instance in session.query(User).order_by(User.id):
    print(instance.name, instance.fullname)

2019-01-20 11:20:22,910 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users ORDER BY users.id
2019-01-20 11:20:22,911 INFO sqlalchemy.engine.base.Engine ()
ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flinstone


## We can also use ORM descriptors as arguments to query - then columns get pulled out 

In [35]:
for name, fullname in session.query(User.name, User.fullname):
    print(name, fullname)

2019-01-20 11:22:20,858 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name, users.fullname AS users_fullname 
FROM users
2019-01-20 11:22:20,859 INFO sqlalchemy.engine.base.Engine ()
ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flinstone


## Query can be limited and offset as well using slicing

In [36]:
for u in session.query(User).order_by(User.id)[1:3]:
    print(u)

2019-01-20 11:24:44,136 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users ORDER BY users.id
 LIMIT ? OFFSET ?
2019-01-20 11:24:44,138 INFO sqlalchemy.engine.base.Engine (2, 1)
<User(name=wendy, fullname=Wendy Williams, password=foobar)>
<User(name=mary, fullname=Mary Contrary, password=xfsdgfsd)>


## Filtering by keyword arguments is done with ```filter_by()```

In [43]:
for name, in session.query(User.name).filter_by(fullname='Ed Jones'): 
    # This always return a tuple so we unpack it - name,
    print(name)

2019-01-20 11:26:53,680 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?
2019-01-20 11:26:53,681 INFO sqlalchemy.engine.base.Engine ('Ed Jones',)
ed


## For filtering using SQL expressions we use ```filter()```

In [44]:
for name, in session.query(User.name).filter(User.fullname=='Ed Jones'):
    print(name)

2019-01-20 11:28:18,154 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?
2019-01-20 11:28:18,156 INFO sqlalchemy.engine.base.Engine ('Ed Jones',)
ed


## The result of a query can be further filtered, we can chain operations

In [45]:
for user in session.query(User).filter(User.name=='ed').filter(User.fullname=='Ed Jones'):
    print(user)

2019-01-20 11:29:35,909 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ? AND users.fullname = ?
2019-01-20 11:29:35,910 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones')
<User(name=ed, fullname=Ed Jones, password=newpass)>


## filter() accepts operators:
- equals -  ==
- not equals - !=
- LIKE - query.filter(User.name.like('%ed%')) - case sensitive like
- ILIKE - query.filter(User.name.ilike('%ed%)) - case insensitive like
- IN - query.filter(User.name.in_(['ed', 'wendy']))
- NOT IN - (Use ~ to negate) query.filter(~User.name.in_(['ed', 'wendy']))
- IS NULL - query.filter(User.name == None) or more pythonic: query.filter(User.name.is_(None))
- IS NOT NULL - query.filtr(USer.name != None) or more pythonic: query.filter(User.name.isnot(None))
- AND - query.filter(and_(User.name=='ed', User.fullname='Ed Jones'))
    we can also just add more expressions to achieve AND operator
    query.filter(User.name=='ed', User.fullname='Ed Jones')
    or chain filters themselves:
    query.filter(User.name=='ed').filter(User.fullname='Ed Jones')
- OR - query.filter(or_(User.name=='ed', User.name=='wendy))
- MATCH - query.filter(User.name.match('wendy')

## Returning lists and scalars

In [48]:
query = session.query(User).filter(User.name.like('%ed')).order_by(User.id)

In [50]:
query.all() # Returns a list of results

2019-01-20 11:43:08,778 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name LIKE ? ORDER BY users.id
2019-01-20 11:43:08,780 INFO sqlalchemy.engine.base.Engine ('%ed',)


[<User(name=ed, fullname=Ed Jones, password=newpass)>,
 <User(name=fred, fullname=Fred Flinstone, password=club)>]

In [51]:
query.first() # Returns the first result in the list of results

2019-01-20 11:43:33,981 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name LIKE ? ORDER BY users.id
 LIMIT ? OFFSET ?
2019-01-20 11:43:33,984 INFO sqlalchemy.engine.base.Engine ('%ed', 1, 0)


<User(name=ed, fullname=Ed Jones, password=newpass)>

In [54]:
user = query.one() # Throws an error if multiple or no rows found.

2019-01-20 11:44:48,835 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name LIKE ? ORDER BY users.id
2019-01-20 11:44:48,836 INFO sqlalchemy.engine.base.Engine ('%ed',)


MultipleResultsFound: Multiple rows were found for one()

## Use ```one_or_none()``` to get ```None``` if no results found (throws error if multiple)

In [55]:
query = session.query(User.id).filter(User.name=='ed').order_by(User.id)

## ```scalar()``` uses ```one()``` on query and returns only the first column in the row

In [56]:
query.scalar() # will return the user.id column only

2019-01-20 11:48:09,115 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id 
FROM users 
WHERE users.name = ? ORDER BY users.id
2019-01-20 11:48:09,117 INFO sqlalchemy.engine.base.Engine ('ed',)


1

# Using textual SQL 